In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import torch
import torch.utils.data as data 

from omegaconf import OmegaConf
from torchvision import transforms
from torch.nn.parallel import DistributedDataParallel as DDP
# 
from contrastive_learning.tests.test_model import load_lin_model, predict_traj_actions
from contrastive_learning.tests.animate_markers import AnimateMarkers
from contrastive_learning.tests.animate_rvec_tvec import AnimateRvecTvec
from contrastive_learning.datasets.dataloaders import get_dataloaders

from contrastive_learning.models.custom_models import LinearInverse
from contrastive_learning.datasets.state_dataset import StateDataset
from contrastive_learning.tests.plotting import plot_rvec_tvec, plot_corners

### Model Loading
Create the distributed group
Load the linear inverse model from the saved path

In [2]:
# Start the multiprocessing to load the saved models properly
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29504"

torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
torch.cuda.set_device(0)

In [3]:
# Set the device and out_dir
device = torch.device('cuda:0')
out_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/out/2022.07.29/16-37_pli_ref_dog_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8'
cfg = OmegaConf.load(os.path.join(out_dir, '.hydra/config.yaml'))
if not ('pos_ref' in cfg):
    cfg.pos_ref = 'global'
model_path = os.path.join(out_dir, 'models/lin_model.pt')

# Load the encoder
lin_model = load_lin_model(cfg, device, model_path)

In [4]:
print(lin_model)

DistributedDataParallel(
  (module): LinearInverse(
    (model): Sequential(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=16, bias=True)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=2, bias=True)
    )
  )
)


In [5]:
print(cfg)

{'agent': {'_target_': 'contrastive_learning.models.agents.pli.PLI', 'loss_fn': 'mse', 'use_encoder': False, 'model': '???', 'optimizer': '???'}, 'optimizer': {'_target_': 'torch.optim.Adam', 'params': '???', 'lr': '???', 'weight_decay': '???'}, 'model': {'_target_': 'contrastive_learning.models.custom_models.LinearInverse', 'input_dim': '???', 'action_dim': '???', 'hidden_dim': '???'}, 'pos_encoder': {'_target_': 'contrastive_learning.models.custom_models.PosToEmbedding', 'input_dim': '???', 'hidden_dim': '???', 'out_dim': '???'}, 'seed': 42, 'device': 'cuda', 'agent_type': 'pli', 'dataset_type': 'state', 'pos_type': 'corners', 'pos_ref': 'dog', 'train_epochs': 1000, 'save_frequency': 10, 'train_dset_split': 0.8, 'batch_size': 64, 'lr': 0.001, 'weight_decay': 1e-05, 'z_dim': 8, 'pos_dim': 8, 'hidden_dim': 64, 'action_dim': 2, 'distributed': True, 'num_workers': 4, 'world_size': 1, 'num_gpus': 4, 'fps': 15, 'frame_interval': 1, 'video_type': 'color', 'experiment': '${agent_type}_ref_${

### Action Animation
1. Dump every predicted action for given data directory
2. Save the predicted and current action in a video

In [6]:
demo_name = 'box_marker_test_2'
exp_name = '{}_{}'.format(out_dir.split('/')[-2], out_dir.split('/')[-1])
data_dir = '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/test_demos/{}'.format(demo_name)
dump_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/tests/animations'
dump_file = '{}_{}.mp4'.format(demo_name, exp_name)

fps = 15

In [7]:
print('dump_file: {}'.format(dump_file))

dump_file: box_marker_test_2_2022.07.29_16-37_pli_ref_dog_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8.mp4


In [8]:
# Get the dataset
dataset = StateDataset(cfg, single_dir=True, single_dir_root=data_dir)
predicted_actions = np.zeros((len(dataset), 2))
test_loader = data.DataLoader(dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=4)

for i, batch in enumerate(test_loader):
    curr_pos, next_pos, action = [b.to(device) for b in batch]
    pred_action = lin_model(curr_pos, next_pos)

    print('Actual Action \t Predicted Action')
    for j in range(len(action)):
        print('{}, \t{}'.format(np.around(dataset.denormalize_action(action[j][0].cpu().detach().numpy()), 2),
                                          dataset.denormalize_action(pred_action[j][0].cpu().detach().numpy())))
        predicted_actions[i*cfg.batch_size+j,:] = dataset.denormalize_action(pred_action[j][0].cpu().detach().numpy())

with open(os.path.join(data_dir, 'predicted_actions.npy'), 'wb') as f:
    np.save(f, predicted_actions)

DATASET POS_REF: dog
len(dataset): 56
self.action_min: [ 0.         -0.30000001], self.action_max: [0.15000001 0.        ]
Actual Action 	 Predicted Action
[0.15 0.  ], 	[0.15877299 0.01754597]
[0.15 0.  ], 	[0.15687031 0.0137406 ]
[0.15 0.  ], 	[0.15570372 0.01140744]
[0.15 0.  ], 	[0.15716417 0.01432833]
[0.15 0.  ], 	[0.15796438 0.01592875]
[0.15 0.  ], 	[ 0.1485647  -0.00287061]
[0.15 0.  ], 	[0.15189894 0.00379787]
[0.15 0.  ], 	[0.15620145 0.01240289]
[ 0.  -0.3], 	[ 0.10847038 -0.08305925]
[ 0.  -0.3], 	[ 0.01745292 -0.26509416]
[0.15 0.  ], 	[ 0.10777617 -0.08444767]
[0.15 0.  ], 	[0.15333682 0.00667362]
[0.15 0.  ], 	[0.15993788 0.01987574]
[0.15 0.  ], 	[0.1575466  0.01509318]
[0.15 0.  ], 	[0.15239663 0.00479325]
[ 0.  -0.3], 	[ 0.07883311 -0.14233379]
[ 0.  -0.3], 	[ 0.04349591 -0.21300819]
[ 0.  -0.3], 	[ 0.04921746 -0.2015651 ]
[0.15 0.  ], 	[ 0.12727907 -0.04544188]
[0.15 0.  ], 	[ 0.13148425 -0.03703151]
[0.15 0.  ], 	[ 0.13394012 -0.03211978]
[0.15 0.  ], 	[ 0.13603535

In [9]:

if cfg.pos_type == 'corners':
    AnimateMarkers(
        data_dir = data_dir, 
        dump_dir = dump_dir, 
        dump_file = dump_file, 
        fps = fps,
        mult_traj = False,
        show_predicted_action = True 
    ) # Saves them in the given dump_file
elif cfg.pos_type == 'rvec_tvec':
    AnimateRvecTvec(
        data_dir = data_dir, 
        dump_dir = dump_dir, 
        dump_file = dump_file,
        fps = fps,
        show_predicted_action=True
    )

Starting Animation


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:08<00:00,  6.57it/s]

Animation saved to: /home/irmak/Workspace/DAWGE/contrastive_learning/tests/animations/box_marker_test_2_2022.07.29_16-37_pli_ref_dog_lf_mse_fi_1_pt_corners_bs_64_hd_64_lr_0.001_zd_8.mp4


### Action Prediction Image
Predict the action for each frame in the test dataset and dump them in a grid image

In [18]:
from copy import deepcopy
cfg.batch_size = 32
global_cfg = deepcopy(cfg)
global_cfg.pos_ref = 'global'
train_loader, test_loader, dataset = get_dataloaders(global_cfg)

DATASET POS_REF: global
len(dataset): 4910
self.action_min: [-0.15000001 -0.30000001], self.action_max: [0.15000001 0.30000001]


In [20]:
len(test_loader.dataset)
batch = next(iter(test_loader))
curr_pos, next_pos, action = [b.to(device) for b in batch]
print('curr_pos: {}, next_pos: {}'.format(curr_pos, next_pos))

# Normalize the current and next pos before inputting to the linear model
ref_tensor = torch.zeros((curr_pos.shape))
half_idx = int(curr_pos.shape[1] / 2) # In order not to have a control for pos_type
if cfg.pos_ref == 'dog':
    ref_tensor = curr_pos[:,half_idx:]
    ref_tensor = ref_tensor.repeat(1,2)
elif cfg.pos_ref == 'box':
    ref_tensor = curr_pos[:,:half_idx]
    ref_tensor = ref_tensor.repeat(1,2)

# curr_pos += ref_tensor
# print('curr_pos: {}'.format(curr_pos))


pred_action = lin_model(curr_pos-ref_tensor, next_pos-ref_tensor)

curr_pos.shape

curr_pos: tensor([[0.2031, 0.4182, 0.2160, 0.3832, 0.2492, 0.3888, 0.2371, 0.4252, 0.1820,
         0.4811, 0.2144, 0.4951, 0.1950, 0.5329, 0.1602, 0.5175],
        [0.3584, 0.3175, 0.3786, 0.2951, 0.4021, 0.3147, 0.3819, 0.3399, 0.3139,
         0.3538, 0.3422, 0.3678, 0.3269, 0.3972, 0.2969, 0.3818],
        [0.4175, 0.0979, 0.3981, 0.1105, 0.3835, 0.0923, 0.4029, 0.0811, 0.3588,
         0.1189, 0.3714, 0.1378, 0.3487, 0.1462, 0.3370, 0.1266],
        [0.3341, 0.1469, 0.3414, 0.1692, 0.3139, 0.1790, 0.3074, 0.1552, 0.3851,
         0.1608, 0.3770, 0.1399, 0.4005, 0.1301, 0.4086, 0.1483],
        [0.5728, 0.2783, 0.5485, 0.2909, 0.5340, 0.2643, 0.5583, 0.2531, 0.5057,
         0.3077, 0.4984, 0.3385, 0.4693, 0.3245, 0.4773, 0.2951],
        [0.2225, 0.1622, 0.1966, 0.1566, 0.2095, 0.1343, 0.2354, 0.1399, 0.2540,
         0.1846, 0.2662, 0.1622, 0.2905, 0.1664, 0.2799, 0.1888],
        [0.3018, 0.3469, 0.2937, 0.3818, 0.2597, 0.3748, 0.2694, 0.3413, 0.2896,
         0.4629, 0.3252, 0.

torch.Size([32, 16])

In [21]:
print(curr_pos[0])

tensor([0.2031, 0.4182, 0.2160, 0.3832, 0.2492, 0.3888, 0.2371, 0.4252, 0.1820,
        0.4811, 0.2144, 0.4951, 0.1950, 0.5329, 0.1602, 0.5175],
       device='cuda:0')


In [22]:
nrows = 4
ncols = int(cfg.batch_size / nrows)
fig, axs = plt.subplots(figsize=(ncols*10,nrows*10), nrows=nrows, ncols=ncols) # Draw the predicted action
print(axs.shape)

for i in range(cfg.batch_size):
    axs_row = int(i / nrows)
    axs_col = int(i % nrows)
    
    action_np = dataset.denormalize_action(action[i].cpu().detach().numpy())
    pred_action_np = dataset.denormalize_action(pred_action[i].cpu().detach().numpy())
    
    if cfg.pos_type == 'corners':
        curr_pos_np = dataset.denormalize_corner(curr_pos[i].cpu().detach().numpy())
        plot_corners(
            ax=axs[axs_col, axs_row],
            curr_pos=curr_pos_np,
            use_img=False,
            img=None,
            plot_action=True,
            actions=(action_np, pred_action_np))
    elif cfg.pos_type == 'rvec_tvec':
        curr_pos_np = dataset.denormalize_pos_rvec_tvec(curr_pos[i].cpu().detach().numpy())
        plot_rvec_tvec(
            ax=axs[axs_col, axs_row],
            curr_pos=curr_pos_np,
            use_img=False,
            img=None,
            plot_action=True,
            actions=(action_np, pred_action_np))
        
plt.savefig(os.path.join(out_dir, 'pil_action_test.jpg'))
    

(4, 8)


In [23]:
print('h')

h
